#Shreyansh Pathak
#D24CSA006
#Assignment 2 , MLOps
#MTech-PhD AI , IITJ

In [2]:
import pandas as pd


day_df = pd.read_csv('day.csv')
hour_df = pd.read_csv('hour.csv')

print(day_df.head())
print("\nHour Dataset:")
print(hour_df.head())
print("\nDay Dataset Summary:")
print(day_df.describe())
print("\nHour Dataset Summary:")
print(hour_df.describe())


   instant      dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1        0        6           0   
1        2  2011-01-02       1   0     1        0        0           0   
2        3  2011-01-03       1   0     1        0        1           1   
3        4  2011-01-04       1   0     1        0        2           1   
4        5  2011-01-05       1   0     1        0        3           1   

   weathersit      temp     atemp       hum  windspeed  casual  registered  \
0           2  0.344167  0.363625  0.805833   0.160446     331         654   
1           2  0.363478  0.353739  0.696087   0.248539     131         670   
2           1  0.196364  0.189405  0.437273   0.248309     120        1229   
3           1  0.200000  0.212122  0.590435   0.160296     108        1454   
4           1  0.226957  0.229270  0.436957   0.186900      82        1518   

    cnt  
0   985  
1   801  
2  1349  
3  1562  
4  1600  

Hour Dataset:
   instant 

In [6]:
!pip install category_encoders


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.8 MB/s eta 0:00:00


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.linear_model import LinearRegression


df = pd.read_csv('day.csv')

df['temp_hum'] = df['temp'] * df['hum']
df['windspeed_temp'] = df['windspeed'] * df['temp']


X = df.drop(['cnt', 'instant', 'dteday', 'casual', 'registered'], axis=1)  # Drop unnecessary columns
y = df['cnt']  # target variable

y = y.astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit']


ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_ohe = pd.DataFrame(ohe.fit_transform(X_train[categorical_features]))
X_test_ohe = pd.DataFrame(ohe.transform(X_test[categorical_features]))


X_train_ohe.columns = X_train_ohe.columns.astype(str)
X_test_ohe.columns = X_test_ohe.columns.astype(str)


X_train_ohe = pd.concat([X_train.reset_index(drop=True).drop(categorical_features, axis=1), X_train_ohe], axis=1)
X_test_ohe = pd.concat([X_test.reset_index(drop=True).drop(categorical_features, axis=1), X_test_ohe], axis=1)


te = TargetEncoder()


X_train_te = X_train.copy()
X_test_te = X_test.copy()
X_train_te[categorical_features] = te.fit_transform(X_train[categorical_features], y_train)
X_test_te[categorical_features] = te.transform(X_test[categorical_features])


X_train_te.columns = X_train_te.columns.astype(str)
X_test_te.columns = X_test_te.columns.astype(str)


model_ohe = LinearRegression()
model_ohe.fit(X_train_ohe, y_train)
y_pred_ohe = model_ohe.predict(X_test_ohe)
mse_ohe = mean_squared_error(y_test, y_pred_ohe)
r2_ohe = r2_score(y_test, y_pred_ohe)

print(f'OneHotEncoder MSE: {mse_ohe}, R-squared: {r2_ohe}')


model_te = LinearRegression()
model_te.fit(X_train_te, y_train)
y_pred_te = model_te.predict(X_test_te)
mse_te = mean_squared_error(y_test, y_pred_te)
r2_te = r2_score(y_test, y_pred_te)

print(f'TargetEncoder MSE: {mse_te}, R-squared: {r2_te}')


class LinearRegressionScratch:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_predicted = np.dot(X, self.weights) + self.bias
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


X_train_te_np = X_train_te.to_numpy()
X_test_te_np = X_test_te.to_numpy()

scratch_model = LinearRegressionScratch(learning_rate=0.01, n_iters=1000)
scratch_model.fit(X_train_te_np, y_train.to_numpy())
y_pred_scratch = scratch_model.predict(X_test_te_np)

mse_scratch = mean_squared_error(y_test, y_pred_scratch)
r2_scratch = r2_score(y_test, y_pred_scratch)

print(f'Scratch Linear Regression MSE: {mse_scratch}, R-squared: {r2_scratch}')


OneHotEncoder MSE: 631635.4866494567, R-squared: 0.8424802921402988
TargetEncoder MSE: 690495.2933771508, R-squared: 0.8278016052134347
Scratch Linear Regression MSE: 999872.4490496456, R-squared: 0.7506479300162043
